# Factor Library Demo

Transparent run: load data, compute factors, show per-step pipeline, analytics, and save outputs.


In [1]:
from datetime import date
import logging
import sys
from pathlib import Path

# Ensure package is importable
try:
    root = Path(__file__).resolve().parents[1]
except NameError:
    root = Path.cwd().resolve().parents[0]
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

from quantlab_factor_library import transforms
from quantlab_factor_library.data_loader import DataLoader
from quantlab_factor_library.factors import Momentum, Volatility, MeanReversion, DollarVolume
from quantlab_factor_library.analytics import (
    compute_all_analytics,
    save_diagnostics,
    compute_factor_correlation,
    save_correlation_matrix,
)
from quantlab_factor_library.run_factors import save_factor
from quantlab_factor_library.paths import factors_dir

logging.basicConfig(level=logging.INFO)
loader = DataLoader(default_end_date=date(2025, 11, 20))
sector_map = None
try:
    sector_map = loader.load_sector_map()
except Exception:
    print("Sector map not available; sector neutralization will be skipped.")


## 1) Load price data & forward returns


In [2]:
price = loader.load_price_wide(dataset="price_daily")
fwd_ret = loader.forward_returns(price)
price.tail()


ticker,A,AA,AAL,AAP,AAPL,ABBV,ABI,ABMD,ABNB,ABT,...,XYL,YHOO,YUM,YUMC,ZBH,ZBRA,ZION,ZMH,ZTS,nan
date,,,,,,,,,,,,,,,,,,,,,
2025-11-14,146.82,37.59,12.76,50.03,272.41,232.36,24.90,NaN,122.02,130.59,...,142.327689,NaN,148.306697,45.941859,89.91,240.56,51.02,NaN,120.82,11.59
2025-11-17,144.52,35.78,12.34,48.93,267.46,234.00,24.92,NaN,117.60,130.03,...,140.592593,NaN,147.321436,46.150821,88.64,232.71,48.92,NaN,118.80,11.52
2025-11-18,143.84,36.55,12.36,49.76,267.44,233.87,24.93,NaN,115.42,130.00,...,139.764933,NaN,148.107655,47.295139,89.08,230.90,49.34,NaN,118.14,11.53
2025-11-19,144.40,36.72,12.33,50.04,268.56,232.91,24.94,NaN,114.44,126.15,...,138.588259,NaN,147.610048,47.474250,89.45,230.93,50.43,NaN,117.08,11.53
2025-11-20,145.06,35.83,12.24,48.37,266.25,229.45,24.94,NaN,111.54,123.97,...,137.650908,NaN,148.495788,47.135930,89.07,228.40,50.44,NaN,115.89,11.54


## 1b) Load Fama-French factors (time series)


In [3]:
ff = loader.load_ff_factors()
ff.head()


,mktrf,smb,hml,rf,umd
date,,,,,
2000-01-03,-0.0071,0.0055,-0.0131,0.0002,-0.0006
2000-01-04,-0.0406,-0.0002,0.0207,0.0002,-0.0191
2000-01-05,-0.0009,0.0031,-0.0005,0.0002,-0.0049
2000-01-06,-0.0074,-0.0043,0.0124,0.0002,-0.0149
2000-01-07,0.0321,-0.004,-0.0157,0.0002,0.0059


## 2) Compute factors (call fac.compute)


In [4]:
factors = [
    Momentum(lookback_days=252, skip_days=21, name="momentum_12m"),
    Volatility(window=60, name="volatility_60d"),
    MeanReversion(lookback_days=5, name="mean_reversion_5d"),
    DollarVolume(window=20, name="dollar_volume_20d"),
]
# Standard compute path
scores = {}
for fac in factors:
    scores[fac.name] = fac.compute(
        loader,
        sector_map=sector_map,
        winsor_limits=(0.01, 0.99),
        min_coverage=0.3,
        fill_method="median",
        neutralize_method="sector",
    )
list(scores.keys())
scores['momentum_12m'].tail()


/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/quantlab_factor_library/factors/volatility.py:20: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = prices.pct_change()
/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/quantlab_factor_library/factors/mean_reversion.py:19: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rev = prices.pct_change(periods=self.lookback_days)


ticker,A,AA,AAL,AAP,AAPL,ABBV,ABI,ABMD,ABNB,ABT,...,XYL,YHOO,YUM,YUMC,ZBH,ZBRA,ZION,ZMH,ZTS,nan
date,,,,,,,,,,,,,,,,,,,,,
2025-11-14,0.014817,-0.169596,-0.119269,1.076401,-0.163753,0.651657,NaN,NaN,-0.362472,0.451993,...,-0.071903,NaN,0.107084,-0.306261,-0.010727,-1.019561,-0.028555,NaN,-0.581512,NaN
2025-11-17,0.065431,-0.116450,-0.232287,0.983886,-0.187480,0.700411,NaN,NaN,-0.363739,0.429659,...,-0.044682,NaN,0.128777,-0.276019,-0.002023,-1.027009,-0.311442,NaN,-0.595210,NaN
2025-11-18,0.026841,-0.177227,-0.296472,0.974084,-0.219670,0.718233,NaN,NaN,-0.264556,0.413250,...,-0.034623,NaN,0.198021,-0.080911,0.010564,-1.057748,-0.193143,NaN,-0.615148,NaN
2025-11-19,0.163778,-0.024089,-0.414601,1.053571,-0.089081,0.819934,NaN,NaN,-0.283902,0.408632,...,-0.017611,NaN,0.273963,-0.002615,0.010052,-1.061389,-0.095946,NaN,-0.654892,NaN
2025-11-20,0.227070,-0.001891,-0.382195,1.116249,-0.098879,0.816348,NaN,NaN,-0.319030,0.326233,...,0.020572,NaN,0.227591,0.010723,-0.009075,-1.027310,-0.048075,NaN,-0.632493,NaN


## 3) Optional - Compute factors (transparent pipeline)


In [5]:
pipeline_scores = {}
for fac in factors:
    raw = fac.post_process(fac.compute_raw_factor(loader))
    filtered = transforms.coverage_filter(raw, min_coverage=0.3)
    wins = transforms.winsorize(filtered, lower=0.01, upper=0.99)
    filled = transforms.fill_factor(wins, method="median")
    neutral = transforms.neutralize_factor(filled, method="sector", sector_map=sector_map)
    z = transforms.zscore(neutral)
    cleaned = transforms.drop_all_nan(z)
    pipeline_scores[fac.name] = cleaned
    print(f"{fac.name}: raw {raw.shape} -> cleaned {cleaned.shape}")
pipeline_scores['momentum_12m'].tail()


momentum_12m: raw (6512, 959) -> cleaned (6238, 959)


/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/quantlab_factor_library/factors/volatility.py:20: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = prices.pct_change()


volatility_60d: raw (6512, 959) -> cleaned (6481, 959)


/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/quantlab_factor_library/factors/mean_reversion.py:19: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rev = prices.pct_change(periods=self.lookback_days)


mean_reversion_5d: raw (6512, 959) -> cleaned (6506, 959)
dollar_volume_20d: raw (6512, 959) -> cleaned (6502, 959)


ticker,A,AA,AAL,AAP,AAPL,ABBV,ABI,ABMD,ABNB,ABT,...,XYL,YHOO,YUM,YUMC,ZBH,ZBRA,ZION,ZMH,ZTS,nan
date,,,,,,,,,,,,,,,,,,,,,
2025-11-14,0.014817,-0.169596,-0.119269,1.076401,-0.163753,0.651657,NaN,NaN,-0.362472,0.451993,...,-0.071903,NaN,0.107084,-0.306261,-0.010727,-1.019561,-0.028555,NaN,-0.581512,NaN
2025-11-17,0.065431,-0.116450,-0.232287,0.983886,-0.187480,0.700411,NaN,NaN,-0.363739,0.429659,...,-0.044682,NaN,0.128777,-0.276019,-0.002023,-1.027009,-0.311442,NaN,-0.595210,NaN
2025-11-18,0.026841,-0.177227,-0.296472,0.974084,-0.219670,0.718233,NaN,NaN,-0.264556,0.413250,...,-0.034623,NaN,0.198021,-0.080911,0.010564,-1.057748,-0.193143,NaN,-0.615148,NaN
2025-11-19,0.163778,-0.024089,-0.414601,1.053571,-0.089081,0.819934,NaN,NaN,-0.283902,0.408632,...,-0.017611,NaN,0.273963,-0.002615,0.010052,-1.061389,-0.095946,NaN,-0.654892,NaN
2025-11-20,0.227070,-0.001891,-0.382195,1.116249,-0.098879,0.816348,NaN,NaN,-0.319030,0.326233,...,0.020572,NaN,0.227591,0.010723,-0.009075,-1.027310,-0.048075,NaN,-0.632493,NaN


## 4) Analytics / diagnostics (IC, autocorr, decile, LS Sharpe/MaxDD, FF reg w/ t-stats)

In [6]:
# Ensure forward returns are available
try:
    fwd_ret
except NameError:
    price = loader.load_price_wide(dataset="price_daily")
    fwd_ret = loader.forward_returns(price)

analytics = {}
ls_returns = {}
ff_regs = {}
for name, s in scores.items():
    res = compute_all_analytics(
        s,
        fwd_ret,
        factor_name=name,
        write_registry=False,
        run_ls_ptf=True,
        top_pct=0.1,
        bottom_pct=0.1,
        ff_factors=ff,
    )
    analytics[name] = res
    if res.get("ls_returns") is not None:
        ls_returns[name] = res["ls_returns"]
    if res.get("ff_regression"):
        ff_regs[name] = res["ff_regression"]

# Quick view of per-factor summary including LS Sharpe/MaxDD
{k: v["summary"] for k, v in analytics.items()}


{'momentum_12m': {'mean_ic': np.float64(0.01048534864223209),
  'ic_tstat': np.float64(5.184340647848556),
  'ic_ir': np.float64(0.06564560542132825),
  'mean_autocorr': np.float64(0.9924280283020469),
  'avg_decile_spread': np.float64(0.014016568929834041),
  'avg_decile_0': 0.001002516642608832,
  'avg_decile_1': 0.0006500515902645419,
  'avg_decile_2': 0.0006285835425994202,
  'avg_decile_3': 0.0006437219487650786,
  'avg_decile_4': 0.000621264311389921,
  'avg_decile_5': 0.0006022979796118529,
  'avg_decile_6': 0.0005625297170332055,
  'avg_decile_7': 0.0006269246738161253,
  'avg_decile_8': 0.0006208195543135337,
  'avg_decile_9': 0.0008174411302989177,
  'ls_return_mean': np.float64(-0.00018478576629497306),
  'ls_return_std': np.float64(0.014399687829020317),
  'ls_sharpe': np.float64(-0.20371157593668285),
  'ls_max_drawdown': np.float64(-0.9177558835697103),
  'ff_alpha': np.float64(-0.00017766437072379843),
  'ff_t_alpha': np.float64(-0.9818437341489391),
  'ff_p_alpha': np.f

## 5) Factor correlation matrix


In [7]:
corr = compute_factor_correlation(scores)
corr_path = save_correlation_matrix(corr) if not corr.empty else None
corr.head(), corr_path


INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_correlation.parquet


(                   momentum_12m  volatility_60d  mean_reversion_5d  \
 momentum_12m           1.000000       -0.030686          -0.013108   
 volatility_60d        -0.030686        1.000000          -0.013522   
 mean_reversion_5d     -0.013108       -0.013522           1.000000   
 dollar_volume_20d      0.016732       -0.017268           0.008073   
 
                    dollar_volume_20d  
 momentum_12m                0.016732  
 volatility_60d             -0.017268  
 mean_reversion_5d           0.008073  
 dollar_volume_20d           1.000000  ,
 PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_correlation.parquet'))

## 7) Save FF factors (time series)


In [8]:
ff_out = factors_dir() / 'factor_ff_timeseries.parquet'
ff.to_parquet(ff_out)
ff_out


PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_ff_timeseries.parquet')

## 8) Save diagnostics and factors


In [9]:
# Diagnostics rows already contain FF betas/t-stats via compute_all_analytics

# Show key LS stats
{k: {key: res['summary'].get(key) for key in ['ls_return_mean','ls_return_std','ls_sharpe','ls_max_drawdown']} for k,res in analytics.items()}

diag_path = save_diagnostics([{'name': name, **res.get('summary', {})} for name, res in analytics.items()])
print("Diagnostics saved to", diag_path)

# Save factors (wide -> long parquet)
factor_paths = {name: save_factor(name, s) for name, s in scores.items()}
print("Factors saved", factor_paths)


INFO:quantlab_factor_library.analytics:Saved diagnostics to /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_step_diagnostics.parquet


Diagnostics saved to /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_step_diagnostics.parquet


INFO:quantlab_factor_library.run_factors:Saved factor momentum_12m to /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_momentum_12m.parquet
INFO:quantlab_factor_library.run_factors:Saved factor volatility_60d to /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_volatility_60d.parquet
INFO:quantlab_factor_library.run_factors:Saved factor mean_reversion_5d to /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_mean_reversion_5d.parquet
INFO:quantlab_factor_library.run_factors:Saved factor dollar_volume_20d to /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_dollar_volume_20d.parquet


Factors saved {'momentum_12m': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_momentum_12m.parquet'), 'volatility_60d': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_volatility_60d.parquet'), 'mean_reversion_5d': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_mean_reversion_5d.parquet'), 'dollar_volume_20d': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_dollar_volume_20d.parquet')}
